# Download data

In [1]:
#!g1.1
main_characters = ['ДЖОУИ', 'МОНИКА', 'РЕЙЧЕЛ', 'РОСС', 'ФИБИ', 'ЧЕНДЛЕР']
labels_to_chars = dict(enumerate(main_characters))
chars_to_labels = {v: k for k, v in labels_to_chars.items()}
labels_to_chars, chars_to_labels

({0: 'ДЖОУИ', 1: 'МОНИКА', 2: 'РЕЙЧЕЛ', 3: 'РОСС', 4: 'ФИБИ', 5: 'ЧЕНДЛЕР'},
 {'ДЖОУИ': 0, 'МОНИКА': 1, 'РЕЙЧЕЛ': 2, 'РОСС': 3, 'ФИБИ': 4, 'ЧЕНДЛЕР': 5})

In [2]:
#!g1.1
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [3]:
#!g1.1
!git clone https://github.com/Alenush/style_transfer_sirius2021summer.git

Cloning into 'style_transfer_sirius2021summer'...
remote: Enumerating objects: 1801, done.
remote: Counting objects: 100% (1801/1801), done.
remote: Compressing objects: 100% (1135/1135), done.
remote: Total 1801 (delta 814), reused 1619 (delta 639), pack-reused 0
Receiving objects: 100% (1801/1801), 52.64 MiB | 12.22 MiB/s, done.
Resolving deltas: 100% (814/814), done.
Checking out files: 100% (657/657), done.


In [4]:
%%bash
cd style_transfer_sirius2021summer
git checkout master
git pull
cd -

Branch 'master' set up to track remote branch 'master' from 'origin'.
Already up to date.
/content


Switched to a new branch 'master'


In [5]:
!pip install rusenttokenize

In [6]:
from rusenttokenize import ru_sent_tokenize

RUSSIAN_SCRIPTS_PATH = 'style_transfer_sirius2021summer/data/scripts/russian'
RUSSIAN_NAMES = [
    'ДЖОУИ',
    'РЕЙЧЕЛ',
    'ФИБИ',
    'МОНИКА',
    'РОСС',
    'ЧЕНДЛЕР'
]
SENTS_BY_NAME = { name : [] for name in RUSSIAN_NAMES }

for name in RUSSIAN_NAMES:
    with open(f'{RUSSIAN_SCRIPTS_PATH}/{name}.txt', 'r') as f:
        lines = f.readlines()[0]

    SENTS_BY_NAME[name] = ru_sent_tokenize(lines)

SENTS_BY_NAME

{'ДЖОУИ': ['Да ладно, ты встречаешься с этим парнем!',
  'С ним должно быть что-то не так!',
  'Вместо?',
  'Никогда не видел этого сна.',
  'Этот парень здоровается, я хочу убить себя.',
  'А ты никогда не знал, что она лесбиянка.',
  'Хорошо, Росс, смотри.',
  'Вы сейчас чувствуете сильную боль.',
  'Ты сердишься.',
  'Тебе больно.',
  'Могу я сказать вам ответ?',
  'Ленточный стык!',
  'Да ладно, ты один!',
  'Есть гормоны!',
  'Я говорю: столкните ее с лестницы.',
  'Эй, тебе что-нибудь нужно, ты всегда можешь прийти к Джоуи.',
  'Мы с Чендлером живем через холл.',
  'И он много раз уезжает.',
  'Что, вроде есть правило или что-то в этом роде?',
  'Кто такой Пол?',
  'Подождите.',
  'Ваше "ненастоящее свидание" сегодня вечером с Полом Винным Парнем?',
  'Привет, Фибс, хочешь помочь?',
  'Что это?',
  'С книжным шкафом покончено!',
  'Росс, позвольте задать вам вопрос.',
  'У нее есть мебель, стереосистема, хороший телевизор - что ты получил?',
  'Вы облажались.',
  'О чем ты говори

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

In [8]:
data = [[], []]

for name in RUSSIAN_NAMES:
    data[0].extend([name] * len(SENTS_BY_NAME[name]))
    data[1].extend(SENTS_BY_NAME[name])

columns = ['Character', 'Phrase']
df_ = pd.DataFrame(data, index=columns).T
# df_ = df_[df_.Phrase.apply(lambda x: len(x.split()) > 3)]
df_.shape

(85356, 2)

In [9]:
df = df_.sample(frac=1).reset_index(drop=True)
df

,Character,Phrase
0,МОНИКА,"Хорошо, я тебя знаю."
1,РОСС,Мне 29.
2,ФИБИ,"О. Ой, Боже, это он!"
3,ЧЕНДЛЕР,Ага!
4,РЕЙЧЕЛ,"Привет, Фиби!"
...,...,...
85351,ДЖОУИ,"Погодите, прежде чем вы мне скажете, что это т..."
85352,РОСС,Тебя взволновала твоя первая ночь вдали от Эммы?
85353,ДЖОУИ,"Это было так круто, когда я был там раньше."
85354,ФИБИ,Ох.


# Preprocessing

In [10]:
!pip install natasha pymorphy2

     |████████████████████████████████| 34.4 MB 31 kB/s 
     |████████████████████████████████| 55 kB 4.4 MB/s 
     |████████████████████████████████| 41 kB 146 kB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
     |████████████████████████████████| 8.2 MB 46.4 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=5d65c5f0717e0d1dba8d5db27252dfc5e4b21250f84fe3e7d539772ae20381c9
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [11]:
import numpy as np

In [12]:
import re
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,

    Doc
)

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

def remove_by_indices(x, indices):
    new_str = ""
    curr_index = 0
    end = 0
    for start, end in indices:
        while curr_index < start:
            new_str += x[curr_index]
            curr_index += 1
        curr_index = end + 1
    new_str += x[end:]
    return new_str

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

def base_form(text: str) -> str:
    char_only = re.sub('[^а-яА-Я ]', '', text) 
    new_str = str(re.sub(' +', ' ', char_only))
    #print(new_str)
    doc = Doc(new_str)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    #return new_str.strip()
    indices = []
    for span in doc.spans:
        if span.type == "PER":
            indices.append((span.start, span.stop))
    without_names = remove_by_indices(new_str, indices)
    new_str = str(re.sub(' +', ' ', without_names))
    lemmatized = ' '.join(lemmatize(new_str))
    return lemmatized.lower().strip()

text = "Миссис Уоллес, я доктор Дрейк Рамори, нейрохирург вашей сестры."
base_form(text)

'я доктор нейрохирург ваш сестра'

In [13]:
df_["Base_Phrase"] = df_["Phrase"].progress_apply(base_form)

100%|██████████| 85356/85356 [07:00<00:00, 202.84it/s]


In [14]:
df_

,Character,Phrase,Base_Phrase
0,ДЖОУИ,"Да ладно, ты встречаешься с этим парнем!",да ладный ты встречаться с это парень
1,ДЖОУИ,С ним должно быть что-то не так!,с они должный быть чтоть не так
2,ДЖОУИ,Вместо?,вместо
3,ДЖОУИ,Никогда не видел этого сна.,никогда не видеть это сон
4,ДЖОУИ,"Этот парень здоровается, я хочу убить себя.",этот парень здороваться я хотеть убить себя
...,...,...,...
85351,ЧЕНДЛЕР,"Но что еще важнее, из-за контроля за арендной ...",но что ещё важный изз контроль за арендный пла...
85352,ЧЕНДЛЕР,Ладно.,ладный
85353,ЧЕНДЛЕР,Все будет хорошо.,всё быть хорошо
85354,ЧЕНДЛЕР,Конечно.,конечно


In [15]:
suspisious_phrases = set(df_["Base_Phrase"].value_counts()[df_["Base_Phrase"].value_counts() > 1].index)
len(suspisious_phrases)

3543

In [16]:
phrase_to_character = dict()
to_del = set()

for index, row in df_.iterrows():
    phrase = row["Base_Phrase"]
    hero = row["Character"]
    if phrase in suspisious_phrases:
        if phrase in phrase_to_character:
            if hero not in phrase_to_character[phrase]:
                phrase_to_character[phrase].append(hero)
        else:
            phrase_to_character[phrase] = [hero]

phrase_to_character

{'смотреть': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'ты больно': ['ДЖОУИ', 'РЕЙЧЕЛ'],
 'кто такой': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'подождать': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'что это': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'о чем ты говорить': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 '': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'поздравлять': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'добрый утро': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'РОСС', 'ЧЕНДЛЕР'],
 'сомневаться': ['ДЖОУИ', 'ЧЕНДЛЕР'],
 'не мочь поверить в то что я здесь слышать': ['ДЖОУИ', 'ФИБИ'],
 'да верно': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'ты серьёзно': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'ЧЕНДЛЕР'],
 'всегда': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС'],
 'ага': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИБИ', 'МОНИКА', 'РОСС', 'ЧЕНДЛЕР'],
 'спасибо': ['ДЖОУИ', 'РЕЙЧЕЛ', 'ФИ

In [17]:
df_vc = df_["Base_Phrase"].value_counts()
df_vc = df_vc[df_vc > 1]
vc = df_vc.to_dict()
vc

{'': 2584,
 'хорошо': 1995,
 'привет': 1990,
 'ага': 1680,
 'нет': 1205,
 'что': 1063,
 'ой': 734,
 'бог мой': 602,
 'да': 475,
 'спасибо': 431,
 'ух ты': 348,
 'знать': 292,
 'отлично': 282,
 'в сам дело': 279,
 'верно': 253,
 'эй': 247,
 'давать': 236,
 'знать что': 219,
 'почему': 218,
 'это': 187,
 'не знать': 181,
 'конечно': 177,
 'угу': 158,
 'хм': 146,
 'всё в порядок': 146,
 'а': 142,
 'какой': 136,
 'ты знать': 134,
 'простить': 131,
 'ого': 130,
 'здравствуйте': 128,
 'я': 127,
 'фиби': 116,
 'пожалуйста': 108,
 'о бог': 100,
 'я тоже': 99,
 'привет ребята': 94,
 'ох': 94,
 'ничего страшный': 94,
 'ладный': 92,
 'о нет': 92,
 'итак': 88,
 'да ладный': 86,
 'что это': 86,
 'как дело': 84,
 'что случиться': 84,
 'пока': 83,
 'извинить': 78,
 'я знать': 76,
 'моника': 75,
 'я жаль': 73,
 'да конечно': 66,
 'что происходить': 65,
 'бог': 64,
 'понятно': 63,
 'нет нет': 63,
 'я очень жаль': 63,
 'подождать': 62,
 'ничего': 58,
 'видеть': 58,
 'действительно': 57,
 'до свидание': 

In [18]:
df_vc = df_vc.reset_index()
df_vc.columns = ['Base_Phrase', 'Count']
df_vc

,Base_Phrase,Count
0,,2584
1,хорошо,1995
2,привет,1990
3,ага,1680
4,нет,1205
...,...,...
3538,я только что спросить,2
3539,я повезти,2
3540,я не знать что с это делать,2
3541,ужин готовый,2


In [19]:
df_vc["Character"] = df_vc["Base_Phrase"].apply(lambda x: phrase_to_character[x])
df_vc

,Base_Phrase,Count,Character
0,,2584,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]"
1,хорошо,1995,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]"
2,привет,1990,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]"
3,ага,1680,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]"
4,нет,1205,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]"
...,...,...,...
3538,я только что спросить,2,"[ФИБИ, МОНИКА]"
3539,я повезти,2,[РЕЙЧЕЛ]
3540,я не знать что с это делать,2,"[ФИБИ, МОНИКА]"
3541,ужин готовый,2,[МОНИКА]


In [20]:
df_vc["Char_count"] = df_vc["Character"].apply(lambda x: len(x))
df_vc

,Base_Phrase,Count,Character,Char_count
0,,2584,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
1,хорошо,1995,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
2,привет,1990,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
3,ага,1680,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
4,нет,1205,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
...,...,...,...,...
3538,я только что спросить,2,"[ФИБИ, МОНИКА]",2
3539,я повезти,2,[РЕЙЧЕЛ],1
3540,я не знать что с это делать,2,"[ФИБИ, МОНИКА]",2
3541,ужин готовый,2,[МОНИКА],1


In [21]:
unconfident_phrases = df_vc[(df_vc.Count >= 4) & (df_vc.Char_count > 2)].reset_index(drop=True)
unconfident_phrases

,Base_Phrase,Count,Character,Char_count
0,,2584,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
1,хорошо,1995,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
2,привет,1990,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
3,ага,1680,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
4,нет,1205,"[ДЖОУИ, РЕЙЧЕЛ, ФИБИ, МОНИКА, РОСС, ЧЕНДЛЕР]",6
...,...,...,...,...
860,где ты черта взять быть,4,"[ДЖОУИ, РЕЙЧЕЛ, МОНИКА, РОСС]",4
861,это быть я,4,"[ДЖОУИ, ФИБИ, МОНИКА]",3
862,что что это,4,"[ДЖОУИ, ФИБИ, РОСС]",3
863,у я есть план,4,"[ДЖОУИ, МОНИКА, ЧЕНДЛЕР]",3


In [22]:
bad_phr = list(unconfident_phrases["Base_Phrase"])
with open("unconfident_base_phrases.txt", "w") as f:
    f.write('\n'.join(bad_phr))

bad_phr

['',
 'хорошо',
 'привет',
 'ага',
 'нет',
 'что',
 'ой',
 'бог мой',
 'да',
 'спасибо',
 'ух ты',
 'знать',
 'отлично',
 'в сам дело',
 'верно',
 'эй',
 'давать',
 'знать что',
 'почему',
 'это',
 'не знать',
 'конечно',
 'угу',
 'хм',
 'всё в порядок',
 'а',
 'какой',
 'ты знать',
 'простить',
 'ого',
 'здравствуйте',
 'я',
 'фиби',
 'пожалуйста',
 'о бог',
 'я тоже',
 'привет ребята',
 'ох',
 'ничего страшный',
 'ладный',
 'о нет',
 'итак',
 'да ладный',
 'что это',
 'как дело',
 'что случиться',
 'пока',
 'извинить',
 'я знать',
 'моника',
 'я жаль',
 'да конечно',
 'что происходить',
 'бог',
 'понятно',
 'нет нет',
 'я очень жаль',
 'подождать',
 'ничего',
 'видеть',
 'действительно',
 'до свидание',
 'что ты делать',
 'ура',
 'ну вот',
 'здесь',
 'хорошо хорошо',
 'ооо',
 'ну ладный',
 'готовый',
 'около',
 'идти',
 'совершенно верно',
 'погодить',
 'вот и всё',
 'ждать',
 'ладный ладный',
 'я в порядок',
 'о чем ты говорить',
 'да да',
 'прекратить',
 'почему бы и нет',
 'ты в п

In [23]:
print(f'Cleaning {int(100 * unconfident_phrases["Count"].sum()/(df.shape[0]))}% of sentences')

Cleaning 32% of sentences


In [24]:
df_clean = df_[~df_["Base_Phrase"].isin(bad_phr)].reset_index(drop=True)
df_clean.columns = ["label", "sentence", "base_sentence"]
df_clean["label"] = df_clean["label"].apply(lambda x: chars_to_labels[x])
df_clean

,label,sentence,base_sentence
0,0,"Да ладно, ты встречаешься с этим парнем!",да ладный ты встречаться с это парень
1,0,С ним должно быть что-то не так!,с они должный быть чтоть не так
2,0,Вместо?,вместо
3,0,Никогда не видел этого сна.,никогда не видеть это сон
4,0,"Этот парень здоровается, я хочу убить себя.",этот парень здороваться я хотеть убить себя
...,...,...,...
57188,5,"Ого, это странно.",ого это странно
57189,5,"Посмотрите вокруг, ребята.",посмотреть вокруг ребята
57190,5,Это был твой первый дом.,это быть твой первый дом
57191,5,"И это было счастливое место, наполненное любов...",и это быть счастливый место наполнить любовь и...


In [25]:
df = df_clean.sample(frac=1).reset_index(drop=True)
df

,label,sentence,base_sentence
0,5,Мамочка?,мамочка
1,4,Плюс мои деньги.,плюс мой деньга
2,5,Там все твои друзья и твои родители!,там всё твой друг и твой родитель
3,2,"Ну да, понимаете что?",ну да понимать что
4,3,"Нет, ребята, я имею в виду, что у меня клавишн...",нет ребята я иметь в вид что у я клавишный до ...
...,...,...,...
57188,0,"Да, и мы пошли бы проверить, но вы забрали наш...",да и мы послать бы проверить но вы забрать наш...
57189,2,Ну так почему бы вам просто не выключить?,ну так почему бы вы просто не выключить
57190,0,"Видишь ли, когда ты играешь, тебе нужно думать...",видеть ли когда ты играть ты нужно думать о та...
57191,3,"Эй, а где Рэйчел?",эй а где


In [26]:
#!g1.1
df.to_csv("only_unique_phrases_ru.csv", index=False)